In [1]:
import time
import os
import sys
import logging

import numpy as np
import tenseal as ts

from sklearn.model_selection import train_test_split
os.chdir("/home/apignet/homomorphic-encryption/ckks_titanic")
from src.features import build_features
from models import encrypted_LR
from models import unencrypted_LR

In [2]:
%load_ext memory_profiler

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
1

1

# definition of parameters

### Paths

In [4]:
#DATA_PATH = "\\data\\raw\\"            # whole data set
DATA_PATH = "/data/quick_demo/"   # subset of the data set, with 15 train_samples and 5 test_samples
#DATA_PATH = "/home/apignet/homomorphic-encryption/ckks_titanic/data/quick_demo/"   # subset of the data set, with 400 train_samples and 50 test_samples
LOG_PATH = "reports/log"
LOG_FILENAME = "test_0716"

In [5]:
os.getcwd()

'/home/apignet/homomorphic-encryption/ckks_titanic'

### Log 

In [6]:
fileHandler = logging.FileHandler("{0}/{1}.log".format(LOG_PATH, LOG_FILENAME))
streamHandler = logging.StreamHandler(sys.stdout)
logging.basicConfig(format="%(asctime)s  [%(levelname)-8.8s]  %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.DEBUG, handlers=[fileHandler, streamHandler])

### Training variables

In [7]:
EPOCH = 20
LEARNING_RATE = 0.8
MOMENTUM_RATE = 0.5
REGULARIZATION_RATE = 0.5
VERBOSE = 2
SAVE_WEIGHT = 2
N_JOBS = 1

# Loading and processing the data

In [8]:
%%memit
logging.info(os.getcwd())
raw_train, raw_test = build_features.data_import(os.getcwd()+DATA_PATH)
train, submission_test = build_features.processing(raw_train, raw_test)
del submission_test

07/23/2020 01:30:02 PM  [INFO    ]  /home/apignet/homomorphic-encryption/ckks_titanic
07/23/2020 01:30:02 PM  [INFO    ]  loading the data into memory (pandas df)
07/23/2020 01:30:02 PM  [INFO    ]  Done
07/23/2020 01:30:02 PM  [INFO    ]  making final data set from raw data
07/23/2020 01:30:02 PM  [INFO    ]  Done
07/23/2020 01:30:02 PM  [INFO    ]  /home/apignet/homomorphic-encryption/ckks_titanic
07/23/2020 01:30:02 PM  [INFO    ]  loading the data into memory (pandas df)
07/23/2020 01:30:02 PM  [INFO    ]  Done
07/23/2020 01:30:02 PM  [INFO    ]  making final data set from raw data
07/23/2020 01:30:02 PM  [INFO    ]  Done
peak memory: 160.84 MiB, increment: 14.11 MiB


In [9]:
%%memit
train, test = train_test_split(train, test_size=0.15)
train_labels = train.Survived
test_labels = test.Survived
train_features = train.drop("Survived", axis=1)
test_features = test.drop("Survived", axis=1)

peak memory: 161.20 MiB, increment: 0.11 MiB


# Definition of safety parameters

# Data encryption

# Initialize the weight

The weights have to be crypted 

In [10]:
%%memit
unencrypted_weight = np.random.normal(loc=0,
                                      scale=0.2, size =(train_features.to_numpy().shape[1]))
logging.info(unencrypted_weight)
unencrypted_bias = np.random.random((1))


07/23/2020 01:30:03 PM  [INFO    ]  [ 0.22344549  0.07938943  0.18219188  0.29127213 -0.12101864  0.0303763
  0.12224563  0.10617969  0.1833412   0.13000056  0.18592974 -0.19709264
  0.01941112 -0.07124375  0.05470446  0.387872   -0.1488696   0.38094098
  0.22940205  0.12254924 -0.0917016  -0.30701546 -0.13310997 -0.1083723
  0.29033805  0.10039627  0.10588454  0.07663559 -0.0617429  -0.06654543
  0.01776944]
07/23/2020 01:30:03 PM  [INFO    ]  [ 0.30799865 -0.16930636 -0.47674358  0.16337647  0.14070674  0.15680225
 -0.37968156  0.17002259  0.20847537 -0.18661598  0.12024911 -0.17646944
  0.25932801  0.26806687  0.29136189  0.05228005  0.36469658 -0.32635753
  0.05472575 -0.17881144  0.07353906 -0.03583613 -0.36988838 -0.27327321
 -0.06129983  0.1573957  -0.07546089 -0.05126098  0.14275369  0.35149886
  0.18916472]
07/23/2020 01:30:03 PM  [INFO    ]  [-0.01636005  0.2388428   0.21856388  0.06373986  0.32920554  0.08945944
 -0.18541534  0.12875049 -0.02091322  0.15083885 -0.09650158 -0

The confidential data as yet stored into a dictionnary, and will be used during the training only by functions which are passed as arguments to the fit methods. This encapsulation of sensitive data will allows us to ensure security during training later. 

# Training the unencrypted model

In [11]:
logging.info("Model initialization")
unencrypted_model = unencrypted_LR.LogisticRegression(init_weight=unencrypted_weight,
                                                      init_bias=unencrypted_bias,
                                                      verbose=VERBOSE,
                                                      save_weight= SAVE_WEIGHT,
                                                      max_epoch=EPOCH,
                                                      learning_rate=LEARNING_RATE,
                                                      momentum_rate=0,
                                                      reg_para=REGULARIZATION_RATE,
                                                      n_jobs = N_JOBS)

07/23/2020 01:30:03 PM  [INFO    ]  Model initialization


In [12]:
logging.info("Training starting")
timer=time.time()
unencrypted_model.fit(train_features.to_numpy(), train_labels.to_numpy().reshape((-1, 1)))
logging.info("Training done. " + str(round(time.time() - timer, 0)) + " seconds")

07/23/2020 01:30:03 PM  [INFO    ]  Training starting
07/23/2020 01:30:03 PM  [INFO    ]  Just finished iteration number 1 


ValueError: cannot reshape array of size 248 into shape (8,1)

In [ ]:
acc = unencrypted_model.accuracy(test_features.to_numpy(), test_labels.to_numpy().reshape((-1, 1)))
logging.info("Accuracy of unencrypted model : %s " % acc)

# Figures

## encrypted and unencrypted losses 

In [ ]:
plt.plot([i*VERBOSE for i in range(len(model.loss_list))], [i.decrypt(secret_key)[0] for i in model.loss_list],'-xr', label='cipherloss')
plt.plot([i*VERBOSE for i in range(len(unencrypted_model.loss_list))], unencrypted_model.loss_list, label='plainloss')
plt.plot([i*VERBOSE for i in range(len(unencrypted_model.true_loss_list))], unencrypted_model.true_loss_list, label='True plainloss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Losses evolution of both models')
plt.legend(loc='best')

## Unencrypted loss

# Errors

## Relative error between losses

In [ ]:
decrypted = [i.decrypt(secret_key)[0] for i in model.loss_list]
true_err_cl = np.abs(np.array(decrypted) - np.array(unencrypted_model.loss_list))
rel_err_cl = true_err_cl/np.abs(decrypted)
true_err_ct = np.abs(np.array(decrypted) - np.array(unencrypted_model.true_loss_list))
rel_err_ct = true_err_ct/np.abs(decrypted)
true_err_lt = np.abs(np.array(unencrypted_model.loss_list) - np.array(unencrypted_model.true_loss_list))
rel_err_lt = true_err_lt/np.abs(unencrypted_model.true_loss_list)


In [ ]:
plt.plot([i*VERBOSE for i in range(len(model.loss_list))],  rel_err_cl, label='cipherloss and plainloss')
plt.plot([i*VERBOSE for i in range(len(model.loss_list))],  rel_err_ct, label='cipherloss and trueloss' )
plt.plot([i*VERBOSE for i in range(len(model.loss_list))],  rel_err_lt, label='plainlosses')
plt.legend(loc='best')
plt.xlabel('Epoch')
plt.ylabel('Relative error on loss')
plt.title('Relative error between losses')

## Relative error between weights

In [ ]:
dwl = np.array([i.decrypt(secret_key) for i in model.weight_list])
dbl = np.array([i.decrypt(secret_key) for i in model.bias_list]).reshape((-1,1))
dl = np.concatenate((dwl,dbl), axis=1)

ul = np.concatenate((np.array(unencrypted_model.weight_list), np.array(unencrypted_model.bias_list).reshape((-1,1))), axis=1)
w_err = np.mean((np.abs(dl-ul)/np.abs(ul)), axis=1)

In [ ]:
plt.plot([i*VERBOSE for i in range(w_err.shape[0])],w_err)
plt.xlabel('Epoch')
plt.ylabel('Relative error on weights')
plt.title('Relative error between encrypted weights and unencrypted weights')

# Error of the log approx, crypted and decrypted

In [ ]:
uX=np.linspace(0.01, 1, 10).reshape((-1,1))
eX = crytp_array(uX, local_context=context)

In [ ]:
def log(x):
    poly_coeff = [-3.57257812,  8.78497194, -6.88562064,  1.81888491]
    return x.polyval([i  for i in poly_coeff])

In [ ]:
el = [log(x) for x in eX]

In [ ]:
dX = [i.decrypt(secret_key) for i in el]

In [ ]:
dX

In [ ]:
np.log(uX)

In [ ]:
np.abs(dX - np.log(uX))/uX